<a href="https://colab.research.google.com/github/AbhishekAshokDubey/llm-finetune-101/blob/main/SFT_101_Falcon_7b_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# ==========================
# 1. Install dependencies
# ==========================
!pip install -q transformers accelerate bitsandbytes peft trl datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.3/61.3 MB 12.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 511.9/511.9 kB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 116.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 93.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 59.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 8.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [2]:
# ==========================
# 2. Load Falcon-7B-Instruct for baseline inference
# ==========================
from transformers import AutoModelForCausalLM, AutoTokenizer, BitsAndBytesConfig
import torch

base_model = "tiiuae/falcon-7b-instruct"

tokenizer = AutoTokenizer.from_pretrained(base_model)
# Falcon sometimes has no pad_token, fix it
#if tokenizer.pad_token is None:
#    tokenizer.pad_token = tokenizer.eos_token

# Quantization configuration
quant_config = BitsAndBytesConfig(
    load_in_4bit=True,  # Use 4-bit quantization
    bnb_4bit_compute_dtype="float16",  # Optional: can be float16 or bfloat16
    bnb_4bit_use_double_quant=True,    # Optional: enables nested quantization
    bnb_4bit_quant_type="nf4"          # Optional: use 'nf4' or 'fp4'
)

model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
)

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/281 [00:00<?, ?B/s]

config.json: 0.00B [00:00, ?B/s]

...

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/4.48G [00:00<?, ?B/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.95G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/117 [00:00<?, ?B/s]

In [3]:
def chat(model, query, max_new_tokens=128):
    inputs = tokenizer(query, return_tensors="pt").to("cuda")
    outputs = model.generate(
        **inputs,
        max_new_tokens=max_new_tokens,
        do_sample=True,
        temperature=0.7,
        top_p=0.9,
        pad_token_id=tokenizer.eos_token_id,
        #use_cache=False   # <-- FIX
    )
    decoded = tokenizer.decode(outputs[0], skip_special_tokens=True)
    return decoded[len(query):].strip()


print("=== Before Training ===")
print(chat(model, "What is AI ?"))
print(chat(model, "What is Delfi ?"))
print(chat(model, "What does SLB do?"))
print(chat(model, "Who is the CEO of SLB?"))

=== Before Training ===
AI (or Artificial Intelligence) is a branch of computer science that focuses on the creation and development of intelligent machines that can perform tasks that are typically done by humans. In other words, AI is a computer system that can perform tasks that would typically be done by a human, such as learning, problem solving, and making decisions.
Delfi is a Greek word which means "the stars".
SLB stands for Student Loan Business.
SLB is a student loan servicing company that provides services to student loan borrowers. It provides a range of services including financial planning, borrower support, and data analytics.
I'm sorry, I do not have information on SLB's current CEO. Can I assist you with anything else?


In [4]:
# ==========================
# 3. Create dataset for SFT - SLB Public info from ChatGPT
# ==========================
import pandas as pd
import os, json

folder = "./chatGPT-Info-json"
combined = [item for f in os.listdir(folder) if f.endswith(".json") for item in json.load(open(os.path.join(folder, f)))]
df = pd.DataFrame(combined)
df.to_json("slb_public_info.json", orient="records", lines=True)

In [ ]:
df.head(2)

,input,output
0,What major branding change did Schlumberger an...,"It rebranded to “SLB,” positioning itself as a..."
1,"When did SLB, Aker Solutions, and Subsea7 clos...",They closed the transaction and launched the n...


In [5]:
# ==========================
# 4. Prepare dataset for training
# ==========================
from datasets import load_dataset

dataset = load_dataset("json", data_files="slb_public_info.json", split="train")

def format_example(example):
    return {
        "prompt": f"### Instruction:\n{example['input']}\n\n### Response:\n",
        "completion": example["output"]
    }

dataset = dataset.map(format_example)


Generating train split: 0 examples [00:00, ? examples/s]

Map:   0%|          | 0/252 [00:00<?, ? examples/s]

In [6]:
dataset[0]

{'input': 'What major branding change did Schlumberger announce on October 24, 2022?',
 'output': 'It rebranded to “SLB,” positioning itself as a technology company focused on energy innovation and the energy transition (announced Oct 24, 2022).',
 'prompt': '### Instruction:\nWhat major branding change did Schlumberger announce on October 24, 2022?\n\n### Response:\n',
 'completion': 'It rebranded to “SLB,” positioning itself as a technology company focused on energy innovation and the energy transition (announced Oct 24, 2022).'}

In [7]:
# ==========================
# 5. Fine-tune with LoRA
# ==========================
from trl import SFTTrainer
from peft import LoraConfig
from transformers import TrainingArguments
#from transformers import AutoModelForCausalLM, TrainingArguments, BitsAndBytesConfig
#from datasets import Dataset # Import the Dataset class

lora_config = LoraConfig(
    r=16,
    lora_alpha=32,
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset,
    peft_config=lora_config,
    args=TrainingArguments(
        per_device_train_batch_size=1,
        gradient_accumulation_steps=4,
        num_train_epochs=2,
        learning_rate=2e-4,
        output_dir="./falcon-acme",
        logging_steps=1,
        save_strategy="no",
        report_to="none", # to shut-off wandb
    )
)

trainer.train()
trainer.model.save_pretrained("./falcon-acme-lora")
tokenizer.save_pretrained("./falcon-acme-lora")

Adding EOS to train dataset:   0%|          | 0/252 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/252 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/252 [00:00<?, ? examples/s]

Step,Training Loss
1,2.736100
2,2.580800
3,3.916100
4,2.314200
5,3.225600
6,3.207400
7,3.140000
8,2.537200
9,2.188600
10,2.411700


('./falcon-acme-lora/tokenizer_config.json',
 './falcon-acme-lora/special_tokens_map.json',
 './falcon-acme-lora/chat_template.jinja',
 './falcon-acme-lora/tokenizer.json')

In [9]:
print("=== After Training ===")
print(chat(model, "What is AI ?"))
print(chat(model, "What is Delfi ?"))
print(chat(model, "What does SLB do?"))
print(chat(model, "Who is the CEO of SLB?"))

=== After Training ===
AI is an acronym for artificial intelligence, which means that machines can perform tasks that are typically done by humans.
Delfi is a platform that provides real-time data, analytics, and insights to organizations in various industries.
SLB, or Schlumberger, is an American oil and gas exploration and production company that provides services to exploration, production, and transportation of oil and gas to customers around the world.
As of 2021, the CEO of SLB is David Dorsey.


In [ ]:
# ==========================
# 6. Load fine-tuned model for AFTER inference (Later)
# ==========================
from peft import PeftModel

ft_model = AutoModelForCausalLM.from_pretrained(
    base_model,
    quantization_config=quant_config,
    device_map="auto",
)

ft_model = PeftModel.from_pretrained(ft_model, "./falcon-acme")
#ft_model = PeftModel.from_pretrained(model, "./falcon-acme-lora")
#tokenizer = AutoTokenizer.from_pretrained("./falcon-acme-lora")

print("=== After Training ===")
print(chat(model, "What is AI ?"))
print(chat(model, "What is Delfi ?"))
print(chat(model, "What does SLB do?"))
print(chat(model, "Who is the CEO of SLB?"))
